In [41]:
from helper_functions import *

In [42]:
s3_client = boto3.client("s3")
ec2_client = boto3.client("ec2")
ssm_client = boto3.client("ssm")

BUCKET_NAME = "laos-datasus"
BUCKET_FOLDER_DBCFILES = "siasus/dbcfiles/"
BUCKET_FOLDER_RAW_TABLES = "siasus/raw_tables/"

In [43]:
ec2_dbc2csv = ec2_client.describe_instances(Filters = [{"Name": "tag:Name", "Values": ["dbc2csv"]}])
ec2_dbc2csv_id = ec2_dbc2csv["Reservations"][0]["Instances"][0]["InstanceId"]

In [ ]:
ec2_client.reboot_instances(InstanceIds=[ec2_dbc2csv_id])

In [44]:
script = \
    "sudo yum update -y && " + \
    "sudo yum install git -y && " + \
    "sudo yum install docker -y && " + \
    "sudo usermod -a -G docker ec2-user && " + \
    "sudo service docker start && " + \
    "sudo chkconfig docker on && " + \
    "sudo curl -L https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m) -o /usr/local/bin/docker-compose && " + \
    "sudo chmod +x /usr/local/bin/docker-compose && " + \
    "sudo yum install python3-pip -y && " + \
    "pip install boto3 && " + \
    "git clone https://github.com/mickaelyoshua7674/aws_create_health_establishments_database.git && " + \
    "newgrp docker"

In [45]:
# RUN SHELL SCRIPT
response_send = ssm_client.send_command(
    DocumentName ="AWS-RunShellScript",
    Parameters = {"commands": ["cd aws_create_health_establishments_database/dbc2csv/ && ls"]},
    InstanceIds = [ec2_dbc2csv_id]
)

# CHECK STATUS OF COMMAND
response_status = ssm_client.get_command_invocation(
    CommandId=response_send["Command"]["CommandId"],
    InstanceId=ec2_dbc2csv_id
)
status = response_status["Status"]
print(status + "...")

while True:
    response_status = ssm_client.get_command_invocation(
        CommandId=response_send["Command"]["CommandId"],
        InstanceId=ec2_dbc2csv_id
    )
    new_status = response_status["Status"]

    if new_status in ["Success", "Cancelled", "Failed", "TimedOut"]: # if command ends
        status = new_status
        print(f"Final status of command is {new_status}.\n")
        break
    elif status == new_status: # if is still the same status
        pass
    else: # if change the status but is not finished
        status = new_status
        print(status + "...")


InProgress...
Final status of command is Success.

